In [58]:
from IPython.display import Image
from IPython.core.display import HTML 

import warnings
warnings.filterwarnings('ignore')

+ flaskのコードの意味などを詳細に

## [DSO] Machine Learning Seminar Vol.8
2020-09-29  
SKUE

In [59]:
Image(url="https://image.yodobashi.com/product/100/000/009/002/935/105/100000009002935105_10204.jpg")

# 機械学習の適用2 - Webアプリケーション
この章で行うこと
+ 訓練済みの機械学習モデルの保存
+ SQLiteの活用
+ Flaskを用いたWebアプリケーションの開発
+ モデルのデプロイ

## 学習済みのscikit-learnの推定器をシリアライズする

第8章で扱ったアウトオブコア学習の結果をpickleで保存しておく。

In [1]:
# import pickle
# import os
# dest = os.path.join('movieclassfier', 'pkl_objects')

# if not os.path.exists(dest):
#     os.makedirs(dest)
    
# pickle.dump(stop,
#                        open(os.path.join(dest, 'stopwords.pkl'), 'wb'),
#                        protocol=4)

# pickle.dump(clf,
#                        open(os.path.join(dest, 'classifier.pkl'), 'wb'),
#                        protocol=4)

## Tips
%%file  
を使えばjupyter notebookで.pyファイルを生成できる。

aaa

In [2]:
%%file movieclassfier/vectorizer.py

from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(os.path.join(cur_dir,
                                            'pkl_objects',
                                            'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + '  '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop ]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                                            n_features=2**21,
                                            preprocessor=None,
                                            tokenizer=tokenizer)

Writing movieclassfier/vectorizer.py


以下はmovieclassfierのディレクトリ下でpythonを立ち上げて実行

In [3]:
# import pickle
# import re
# import os
# from vectorizer import vect
# clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))


# import numpy as np
# label = {0:'negative',
#               1:'positive'}

# example = ['I love this movie']
# X = vect.transform(example)
# print('Prediction: %s\nProbability: %.2f%%' \
#       %(label[clf.predict(X)[0]],np.max(clf.predict_proba(X))*100))

## データストレージとしてSQLiteデータベースを設定する
+ SQLiteとは
 + OSSのSQLデータベース
 + サーバは用意しなくてもいい。
 + 全てが1つのファイルに含まれたデータベースと考えることができる。

テーブルを生成して、レコードを2件追加する。

In [7]:
import sqlite3
import os

if os.path.exists('reviews.sqlite'):
    os.remove('reviews.sqlite')
    
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT) ')
example1 = 'I love this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES"\
                     "(?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES"\
                     "(?, ?, DATETIME('now'))", (example2, 0))
conn.commit()
conn.close()

SQLiteに保存したデータを読み込む

In [8]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute("SELECT * FROM review_db WHERE date"\
                   " BETWEEN '2017-01-01 00:00:00' AND DATETIME('now') ")
results = c.fetchall()
conn.close()
print(results)

[('I love this movie', 1, '2020-09-19 08:27:24'), ('I disliked this movie', 0, '2020-09-19 08:27:24')]


## Flaskを使ってWebアプリケーションを開発する
+ Flaskとは
 + 軽量なWebアプリケーションのフレームワーク

In [10]:
# !pip install flask

In [12]:
# !mkdir 1st_flask_app_1

In [14]:
# !mkdir 1st_flask_app_1/templates

In [15]:
%%file 1st_flask_app_1/app.py

from flask import Flask, render_template

app = Flask(__name__) # 初期化して、テンプレが同じディレクトリ内で見つかるようにしている。
@app.route('/') # インデックス関数が実行すべきURLを指定している。

def index():
    return render_template('first_app.html') # レンダリング

if __name__ == '__main__':
    app.run() # サーバ上のアプリケーションを実行

Writing 1st_flask_app_1/app.py


In [17]:
%%file 1st_flask_app_1/templates/first_app.html
<!doctype html>
<html>
    <head>
        <title>First app</title>
    </head>
        <div>Hi, this is my first Flaskweb app!</div>
    </body>
</html>

Overwriting 1st_flask_app_1/templates/first_app.html


In [20]:
# !pip install wtforms

In [21]:
# !mkdir 1st_flask_app_2

In [23]:
# !mkdir 1st_flask_app_2/static

In [24]:
# !mkdir 1st_flask_app_2/templates

In [30]:
%%file 1st_flask_app_2/app.py

from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators

app = Flask(__name__)

class HelloForm(Form):
    sayhello = TextAreaField(' ', [validators.DataRequired()]) # テキストフィールドの拡張
    
@app.route('/')
def index():
    form = HelloForm(request.form)
    return render_template('first_app.html', form=form)

@app.route('/hello', methods=['POST'])
def hello():
    form = HelloForm(request.form)
    if request.method == 'POST' and form.validate(): # POSTメソッドを使ってフォームのデータをサーバに送信する。
        name = request.form['sayhello']
        return render_template('hello.html', name=name)
    
    return render_template('first_app.html', form=form)

if __name__ == '__main__':
    app.run(debug=True)

Overwriting 1st_flask_app_2/app.py


In [35]:
%%file 1st_flask_app_2/templates/_formhelpers.html

{% macro render_field(field) %}
    <dt>{{ field.label  }}
    <dd>{{ field(**kwargs) | safe }}
    {% if field.errors %}
        <ul class=errors>
        {% for error in field.errors %}
            <li>{{  error  }}</li>
        {% endfor %}
        </ul>
    {% endif %}
    </dd>
    </dt>
{% endmacro %}

Writing 1st_flask_app_2/templates/_formhelpers.html


In [36]:
%%file 1st_flask_app_2/static/style.css

body{
    font-size: 2em;
}

Overwriting 1st_flask_app_2/static/style.css


In [37]:
%%file 1st_flask_app_2/templates/first_app.html
<!doctype html>
<html>
    <head>
        <titile>First app</title>
        <link rel="stylesheet"
                    href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
    {% from "_formhelpers.html" import render_field %}
    
    <div>Whats your name?</div>
    <form method=post action="/hello">
        <dl>
            {{ render_field(form.sayhello) }}
        </dl>
        <input type=submit value='Say Hello' name='submit_btn'>
    </form>
    </body>
</html>

Overwriting 1st_flask_app_2/templates/first_app.html


In [38]:
%%file 1st_flask_app_2/templates/hello.html
<!doctype html>
<html>
    <head>
        <titile>First app</title>
        <link rel="stylesheet"
                    href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
        <div>Hello {{ name }}</div>
    </body>
</html>

Overwriting 1st_flask_app_2/templates/hello.html


In [57]:
%%file movieclassfier/app.py

from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
import numpy as np

# ローカルディレクトリからHashingVectorizerをインポート
from vectorizer import vect

# ローカルディレクトリからupdate関数をインポート
from update import update_model

app = Flask(__name__)

####### 分類器の準備
cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir, 'pkl_objects', 'classifier.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

def classify(document):
    label = {0: 'negative',
                  1: 'positive'}
    X = vect.transform([document])
    y = clf.predict(X)[0]
    proba = clf.predict_proba(X).max()
    return label[y], proba

def train(document, y):
    X = vect.transform([document])
    clf.partial_fit(X, [y])
    
def sqlite_entry(path, document, y):
    conn = sqlite3.connect(path)
    c = conn.cursor()
    c.execute("INSERT INTO review_db (review, sentiment, date)"\
                        " VALUES (?, ?, DATETIME('now'))", (document, y))
    conn.commit()
    conn.close()
    
    
###### Flask
class ReviewForm(Form):
    moviereview = TextAreaField(' ',
                                                           [validators.DataRequired(),
                                                               validators.length(min=15)]) # レビューは15文字以内
    
@app.route('/')
def index():
    form = ReviewForm(request.form)
    return render_template('reviewform.html', form=form)

@app.route('/requests', methods=['POST'])
def results():
    form = ReviewForm(request.form)
    if request.method == 'POST' and form.validate():
        review = request.form['moviereview']
        y, proba = classify(review)
        return render_template('results.html',
                                                  contents=review,
                                                  prediction=y,
                                                  probability=round(proba*100, 2))
    return render_template('reviewform.html', form=form)

@app.route('/thanks', methods=['POST'])
def feedback():
    feedback = request.form['feedback_button']
    review = request.form['review']
    prediction = request.form['prediction']
    
    inv_label = {'negative':0,
                          'positive':1}
    y = inv_label[prediction]
    if feedback == 'Incorrect':
        y = int(not(y))
    train(review, y)
    sqlite_entry(db, review, y)
    return render_template('thanks.html')

if __name__ == '__main__':
    clf = update_model(db_path=db, model=clf, batch_size=10000)
    app.run(debug=True)

Overwriting movieclassfier/app.py


In [43]:
# !mkdir movieclassfier/templates

In [50]:
%%file movieclassfier/templates/reviewform.html
<!doctype html>
<html>
    <head>
        <link rel="stylesheet"
                    href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
        <h2>Please enter your movie review:</h2>
        
        {% from "_formhelpers.html" import render_field %}
        
        <form method=post action="/requests">
            <dl>
                {{  render_field(form.moviereview, cols='30', rows='10') }}
            </dl>
            <div>
                <input type=submit value='Submit review' name='submit_btn'>
            </div>
        </form>
    </body>
</html>

Overwriting movieclassfier/templates/reviewform.html


In [49]:
%%file movieclassfier/templates/results.html
<!doctype html>
<html>
    <head>
        <titile>Movie Classification</title>
        <link rel="stylesheet"
                    href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
        <h3>Your movie review:</h3>
            <div>{{ content }}</div>
            
        <h3>Prediction</h3>
            <div>This movie review is <strong>{{ prediction }}</strong>
                (probability: {{ probability }}%).</div>
                
            <div id='button'>
                <form action="/thanks" method="post">
                    <input type=submit value='Correct' name='feedback_button'>
                    <input type=submit value='Incorrect' name='feedback_button'>
                    <input type=hidden value='{{ prediction }}' name='prediction'>
                    <input type=hidden value='{{ content }}' name='review'>
                </form>
            </div>
            
            <div id='button'>
                <form action="/">
                    <input type=submit value='Submit another review'>
                </form>
            </div>
    </body>
</html>

Overwriting movieclassfier/templates/results.html


In [47]:
# !mkdir movieclassfier/static

In [48]:
%%file movieclassfier/static/style.css

body{
    width:600px;
}
.button{
    padding-top: 20px;
}

Writing movieclassfier/static/style.css


In [51]:
%%file movieclassfier/templates/thanks.html
<!doctype html>
<html>
    <head>
        <titile>Movie Classification</title>
        <link rel="stylesheet"
                    href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
        <h3>Thank you for your feedbacks!</h3>
            <div id='button'>
                <form action="/">
                    <input type=submit value='Submit another review'>
                </form>
            </div>
    </body>
</html>

Writing movieclassfier/templates/thanks.html


In [52]:
%%file movieclassfier/templates/_formhelpers.html

{% macro render_field(field) %}
    <dt>{{ field.label  }}
    <dd>{{ field(**kwargs) | safe }}
    {% if field.errors %}
        <ul class=errors>
        {% for error in field.errors %}
            <li>{{  error  }}</li>
        {% endfor %}
        </ul>
    {% endif %}
    </dd>
    </dt>
{% endmacro %}

Writing movieclassfier/templates/_formhelpers.html


## PythonAnywhereでのサーバ上でのアプリケーション実行

/home/kamonohashiperio/movieclassifier/app.py  

virtualenvでsklearnを入れないと動かない。

mkvirtualenv periove --python=/usr/bin/python3.8

https://www.pythonanywhere.com/forums/topic/14032/

### PythonAnywhereでの手順
+ 1.
+ 2.
+ 3.

In [56]:
%%file movieclassfier/update.py

import pickle
import sqlite3
import numpy as np
import os

# ローカルディレクトリからHashingVectorizerをインポート
from vectorizer import vect

def update_model(db_path, model, batch_size=10000):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('SELECT * from review_db')
    
    results = c.fetchmany(batch_size)
    while results:
        data = np.array(results)
        X = data[:, 0]
        y = data[:, 1].astype(int)
        classes = np.array([0, 1])
        X_train = vect.transform(X)
        model.partial_fit(X_train, y, classes=classes)
        results = c.fetchmany(batch_size)
        
    conn.close()
    return model

cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir, 'pkl_objects', 'classifier.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

clf = update_model(db_path=db, model=clf, batch_size=10000)

# classifier.pklのファイルを永続的に更新したい場合は以下のコードのコメントを解除
# pickle.dump(clf, open(os.path.join(cur_dir, 'pkl_objects', 'classifier.pkl'), 'wb'), protocol=4)

Overwriting movieclassfier/update.py
